# 乗降客数

In [ ]:
stations = pd.read_csv('input/stations_tokai3ken_edited.csv', index_col=0)

In [ ]:
a_23 = stations[stations['路線ID'] == 'a_23.0'].sort_values(by='乗降客数（日）')
m_2 = stations[stations['路線ID'] == 'm_2.0'].sort_values(by='乗降客数（日）')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(a_23["乗降客数（日）"], a_23["飲食店事業所数"], 'o-')
#plt.plot(a_23["乗降客数（日）"]/10000, a_23["飲食店事業所数"], 'o-')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(m_2["乗降客数（日）"], m_2["飲食店事業所数"], 'o-')

# ここから

In [ ]:
import pystan

In [ ]:
stations = pd.read_csv('input/stations_tokai3ken_edited.csv', index_col=0)

In [ ]:
#stations = stations.query('路線ID == ["a_23.0", "m_2.0"]')
#stations = stations.query('路線ID == "a_23.0"')

In [ ]:
stations

In [ ]:
X = pd.concat([stations['乗降客数（日）'], stations['路線ID']], axis=1)
y = stations['飲食店事業所数']

In [ ]:
#X['乗降客数（日）'] = X['乗降客数（日）']/X['乗降客数（日）'].max()
#y = y/y.max()

In [ ]:
#X['乗降客数（日）'] = X['乗降客数（日）']/10000

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
#多治見、土岐、(瑞浪、)恵那、中津川をtestに移動
X_tono = pd.DataFrame(data=[X_train.loc['g_1189'], X_train.loc['g_1190'], X_train.loc['g_1194'], X_train.loc['g_1196']])
X_test = pd.concat([X_test, X_tono], axis=0)
X_train = X_train.drop(['g_1189', 'g_1190', 'g_1194', 'g_1196'], axis=0)
y_tono = pd.Series(data=[y_train.loc['g_1189'], y_train.loc['g_1190'], y_train.loc['g_1194'], y_train.loc['g_1196']],
                      index=['g_1189', 'g_1190', 'g_1194', 'g_1196'])
y_test = pd.concat([y_test, y_tono], axis=0)
y_train = y_train.drop(['g_1189', 'g_1190', 'g_1194', 'g_1196'], axis=0)

In [ ]:
#X_train

In [ ]:
rosen = dict(zip(stations['路線ID'].unique(), range(1, 1+len(stations['路線ID'].unique()))))
Rosen_train = []
Rosen_test = []
for i in range(len(X_train)):
    Rosen_train.append(rosen[X_train['路線ID'][i]])
for i in range(len(X_test)):
    Rosen_test.append(rosen[X_test['路線ID'][i]])

data = dict(
    N_train=X_train.shape[0],
    N_test=X_test.shape[0],
    N_rosen=len(rosen),
    Joko_train=X_train['乗降客数（日）'].values,
    Joko_test=X_test['乗降客数（日）'].values,
    Rosen_train=Rosen_train,
    Rosen_test=Rosen_test,
    Inshoku_train=y_train.values
)

In [ ]:
model = '''
data {
    int N_train;
    int N_test;
    int N_rosen;
    vector<lower=0>[N_train] Joko_train;
    vector<lower=0>[N_test] Joko_test;
    int<lower=1> Rosen_train[N_train];
    int<lower=1> Rosen_test[N_test];
    vector<lower=0>[N_train] Inshoku_train;
}

parameters {
    real mu_a;
    real<lower=0> mu_b;
    real<lower=0> mu_s;
    real<lower=0> s_a;
    real<lower=0> s_b;
    real<lower=0> s_s;
    vector<lower=0, upper=50>[N_rosen] a;
    vector<lower=0, upper=0.01>[N_rosen] b;
    vector<lower=0, upper=25>[N_rosen] s;
#    real<lower=0> s;
}

model {
#    mu_a ~ uniform(0, 50);
#    mu_b ~ uniform(0, 0.01);
    a ~ normal(mu_a, s_a);
    b ~ normal(mu_b, s_b);
    s ~ normal(mu_s, s_s);
#    a ~ uniform(0, 50);
#    b ~ uniform(0, 0.01);
#    s ~ uniform(0, 25);
    for (i in 1:N_train)
#        Inshoku_train[i] ~ normal(a[Rosen_train[i]] + b[Rosen_train[i]]*Joko_train[i], s[Rosen_train[i]]);
#        Inshoku_train[i] ~ normal(a[Rosen_train[i]] + b[Rosen_train[i]]*Joko_train[i], s);
        Inshoku_train[i] ~ student_t(4, a[Rosen_train[i]] + b[Rosen_train[i]]*Joko_train[i], s[Rosen_train[i]]);
}

generated quantities{
    vector[N_test] predict;
    for (i in 1:N_test)
#        predict[i] = normal_rng(a[Rosen_test[i]] + b[Rosen_test[i]]*Joko_test[i], s[Rosen_test[i]]);
#        predict[i] = normal_rng(a[Rosen_test[i]] + b[Rosen_test[i]]*Joko_test[i], s);
        predict[i] = student_t_rng(4, a[Rosen_test[i]] + b[Rosen_test[i]]*Joko_test[i], s[Rosen_train[i]]);
}
'''

In [ ]:
#fit = pystan.stan(model_code=model, data=data, chains=3, iter=500, warmup=100, thin=1)
fit = pystan.stan(model_code=model, data=data, chains=4, iter=2000, warmup=500, thin=1)

In [ ]:
fit

In [ ]:
#fit.extract()['predict'][0]

In [ ]:
#y_test

In [ ]:
predict = fit.extract()['predict'][0]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
#ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

In [ ]:
#MCMCサンプリングの結果を抽出
ms = fit.extract(permuted=False, inc_warmup=True)
#ウォームアップ（バーンイン）のサイズを取得
iter_from = fit.sim['warmup']
#ウォームアップの区間を省く
iter_range = np.arange(iter_from, ms.shape[0])
#各変数名を取得
paraname = fit.sim['fnames_oi']

#※※※今回は全て描画したいので、こちらを使う
iter_start = np.arange(0, ms.shape[0])

In [ ]:
#seabornのcolorpalette
palette = sns.color_palette()
#おまじない？
sns.set(font_scale=1)
sns.set_style("ticks")
sns.despine(offset=10, trim=True)

#複数グラフの描画（これしか方法知らない）
fig,axes  = plt.subplots(nrows=2, ncols=3, figsize=(15,10))

for i in range(2):
    for j in range(3):
        axes[i,j].plot(iter_start, ms[iter_start, :, i*2+j], 
                       linewidth=2, color=palette[i*2+j])
        axes[i,j].set_title(paraname[i*2+j])
        axes[i,j].set_xlabel('mcmc_size')
        axes[i,j].set_ylabel('parameter')
        axes[i,j].grid(True)

fig.show()

In [ ]:
plt.hold(True);
a = fit.extract()['a'][0, 0]
b = fit.extract()['b'][0, 0]
xx = range(250000)
yy = a + b*xx
plt.plot(xx, yy)
plt.plot(a_23["乗降客数（日）"], a_23["飲食店事業所数"], 'o-')

In [ ]:
plt.hold(True);
a = fit.extract()['a'][0, 1]
b = fit.extract()['b'][0, 1]
xx = range(7000)
yy = a + b*xx
plt.plot(xx, yy)
plt.plot(m_2["乗降客数（日）"], m_2["飲食店事業所数"], 'o-')

一応それっぽくa, bが求めれてるのもあるけど、全体の精度ごみ

データが悪かった（分散大きすぎ）から？